# 测试
## 使用复旦大学项目测试api

In [1]:
import requests
from click import prompt

org = "OpenMOSS"

url = f"https://api.github.com/orgs/{org}/repos?per_page=100"
headers = {"Accept": "application/vnd.github+json"}  # 可加 Authorization 提高速率限制

response = requests.get(url, headers=headers)
repos = response.json()

context = []

for repo in repos:
    if int(repo["stargazers_count"]) >= 50:
        context.append({
            "name": repo["name"],
            "description": repo.get("description"),
            "created_at": repo.get("created_at"),
            "updated_at": repo.get("updated_at"),
            "stars": repo.get("stargazers_count",0),
            "url": repo.get("html_url")
        })



In [2]:
context

[{'name': 'CoLLiE',
  'description': 'Collaborative Training of Large Language Models in an Efficient Way',
  'created_at': '2023-04-02T11:10:19Z',
  'updated_at': '2025-08-20T07:08:09Z',
  'stars': 416,
  'url': 'https://github.com/OpenMOSS/CoLLiE'},
 {'name': 'MOSS',
  'description': 'An open-source tool-augmented conversational language model from Fudan University',
  'created_at': '2023-04-15T14:07:44Z',
  'updated_at': '2025-09-06T03:26:11Z',
  'stars': 12060,
  'url': 'https://github.com/OpenMOSS/MOSS'},
 {'name': 'HalluQA',
  'description': 'Dataset and evaluation script for "Evaluating Hallucinations in Chinese Large Language Models"',
  'created_at': '2023-10-04T03:01:40Z',
  'updated_at': '2025-08-07T12:29:33Z',
  'stars': 132,
  'url': 'https://github.com/OpenMOSS/HalluQA'},
 {'name': 'Say-I-Dont-Know',
  'description': "[ICML'2024] Can AI Assistants Know What They Don't Know?",
  'created_at': '2024-01-18T03:18:26Z',
  'updated_at': '2025-08-14T01:24:31Z',
  'stars': 83,
  

In [3]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
os.environ["HTTP_PROXY"] = "http://127.0.0.1:8890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:8890"
# ucloud api key
API_KEY = os.getenv("UCLOUD_API_KEY")
API_BASE = "https://api.modelverse.cn/v1"
model = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    model="deepseek-ai/DeepSeek-V3-0324")
output_parser = StrOutputParser()


In [4]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template="""

请根据json格式数据中description的内容分类到这些标签，最后更新时间（updated_at)2023年之前的项目请省略，如果输入jason格式为空请跳过一下内容，直接输出“此组织在GitHub上无star数大于50的项目”：

模型（如大语言模型，多模态模型等）
数据集（各类database或者dataset）
工具（各种可用于辅助的ai工具）
其他

jason数据为：

{context}

请逐个输出每个项目的分类，并以star数量排序（最多star数量的在最前面），


输出格式按照：

**项目名**
分类：
star数字：
最后更新时间：
项目描述：(用你自己的话总结一下）
url:

在输出前确认一下格式以及是否有按star数排序
""")

chain = prompt | model | output_parser
result = chain.invoke({"context":context})
print(result)

以下是按照star数量排序后的项目分类结果，仅包含star数大于50且最后更新时间在2023年及之后的项目：

---

**MOSS**  
分类：模型  
star数字：12060  
最后更新时间：2025-09-06  
项目描述：复旦大学开源的增强工具对话语言模型。  
url: https://github.com/OpenMOSS/MOSS  

---

**MOSS-TTSD**  
分类：模型  
star数字：941  
最后更新时间：2025-09-05  
项目描述：支持中英文表达性对话语音合成的模型，具备零样本多说话人克隆和长文本生成能力。  
url: https://github.com/OpenMOSS/MOSS-TTSD  

---

**AnyGPT**  
分类：模型  
star数字：861  
最后更新时间：2025-09-05  
项目描述：基于离散序列建模的统一多模态大语言模型。  
url: https://github.com/OpenMOSS/AnyGPT  

---

**CoLLiE**  
分类：工具  
star数字：416  
最后更新时间：2025-08-20  
项目描述：高效协作训练大语言模型的方法。  
url: https://github.com/OpenMOSS/CoLLiE  

---

**SpeechGPT-2.0-preview**  
分类：模型  
star数字：354  
最后更新时间：2025-09-02  
项目描述：达到GPT-4o水平的实时口语对话系统。  
url: https://github.com/OpenMOSS/SpeechGPT-2.0-preview  

---

**VLABench**  
分类：数据集  
star数字：283  
最后更新时间：2025-09-04  
项目描述：用于公平评估视觉语言代理（VLA）、具身智能体和多模态模型的大规模基准。  
url: https://github.com/OpenMOSS/VLABench  

---

**Language-Model-SAEs**  
分类：工具  
star数字：146  
最后更新时间：2025-08-30  
项目描述：专注于稀疏自编码器（SAE）在语言模型可解

In [5]:
with open(f"Github/{org}_results.txt", "w", encoding="utf-8") as file:
    file.write(result)

# 生产
## 从Github上搜索信息

In [48]:
github_orgs = """
FudanDISC
OpenMOSS
fudan-generative-vision
Thinklab-SJTU
VISION-SJTU
USTCAGI
ECNU-CILAB
opengvlab
stepfun-ai
minimax-ai
OpenSenseNova
loongOpen
AgibotTech
FFTAI
4paradigm
TigerResearch
infly-ai
IAAR-Shanghai
MemTensor
infinigence
"""

In [45]:
from tqdm import tqdm


webs = github_orgs.strip().split("\n")

for org in tqdm(webs, total=len(webs)):
    url = f"https://api.github.com/orgs/{org}/repos?per_page=100"
    headers = {"Accept": "application/vnd.github+json"}  # 可加 Authorization 提高速率限制

    response = requests.get(url, headers=headers)
    repos = response.json()

    context = []

    for repo in repos:
        try:
            if int(repo["stargazers_count"]) >= 50:
                context.append({
                    "name": repo["name"],
                    "description": repo["description"],
                    "created_at": repo["created_at"],
                    "updated_at": repo["updated_at"],
                    "stars": repo["stargazers_count"],
                    "url": repo["html_url"]
                })
        except:
            pass

    result = chain.invoke({"context":context})
    with open(f"Github/{org}_results.txt", "w", encoding="utf-8") as file:
        file.write(result)

100%|██████████| 17/17 [05:33<00:00, 19.60s/it]


[{'name': 'R3Det_Tensorflow',
  'description': 'Code for AAAI 2021 paper: R3Det: Refined Single-Stage Detector with Feature Refinement for Rotating Object',
  'created_at': '2019-08-27T14:42:38Z',
  'updated_at': '2025-03-06T05:52:31Z',
  'stars': 546,
  'url': 'https://github.com/Thinklab-SJTU/R3Det_Tensorflow'},
 {'name': 'ThinkMatch',
  'description': 'A research protocol for deep graph matching.',
  'created_at': '2019-09-06T17:07:08Z',
  'updated_at': '2025-08-31T14:47:22Z',
  'stars': 872,
  'url': 'https://github.com/Thinklab-SJTU/ThinkMatch'},
 {'name': 'CSL_RetinaNet_Tensorflow',
  'description': 'Code for ECCV 2020 paper: Arbitrary-Oriented Object Detection with Circular Smooth Label',
  'created_at': '2020-03-07T08:48:15Z',
  'updated_at': '2025-06-13T20:11:34Z',
  'stars': 192,
  'url': 'https://github.com/Thinklab-SJTU/CSL_RetinaNet_Tensorflow'},
 {'name': 'S2TLD',
  'description': 'Newly released traffic light dataset for small object detection.',
  'created_at': '2020-04